# Random Perturbations

Streamlined notebook for evaluating random perturbations on saved models and datasets.

## Imports

In [1]:
# Standard library
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.perturb_funcs import ( analyze_wiggles_metrics )

from minima_volume.dataset_funcs import (
    load_dataset,
    load_model,
    load_models_and_data,
    prepare_datasets,
    tensor_to_list,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:
# Perturbation Configuration
perturbation_seed = 1
num_directions = 500
N = 100
x = np.linspace(0, 1, N)
coefficients = x**2

# Other Configuration 
dataset_quantities = [0, (int)(0.08*9409), 
                      (int)(0.18*9409), (int)(0.28*9409),
                      (int)(0.38*9409), (int)(0.48*9409),
                     (int)(0.58*9409), (int)(0.68*9409),] # modulo arithmetic dataset sizes
base_output_dir = ""

## Model + Dataset Specific Code

This is for model and dataset specific code.

In [3]:
# User specifies the model module name
import modulo_arithmetic_model_data as model_module
modulus = 97

# MNIST specific initialization parameters
hidden_dims = [250]

# Grab model
model_template = model_module.get_model(N = modulus, hidden_dims=hidden_dims, device=device, seed=0)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Loading Model and Datasets

In [4]:
# ====================================
# Load Trained Models and Dataset
# ====================================
target_dir = "models_and_data"  # relative path
loaded_models, loaded_model_data, loaded_dataset = load_models_and_data(
    model_template=model_template,
    target_dir=target_dir,
    device=device,
)

# Dataset Info
dataset_type = loaded_dataset['dataset_type']
print(f"Dataset type: {dataset_type}")
print(f"Dataset quantities: {loaded_dataset['dataset_quantities']}")

print("\nTensor shapes:")
for key in ["x_base_train", "y_base_train", "x_additional", "y_additional", "x_test", "y_test"]:
    shape = getattr(loaded_dataset[key], "shape", None)
    print(f"  {key}: {shape if shape is not None else 'None'}")

# Reconstruct trained_model dicts safely.
# If the loss or accuracy or additional metrics happen to be
# tensors, they get safely converted to lists.
all_models = [
    {
        "model": model,
        **{
            k: tensor_to_list(model_data[k], key_path=k)
            for k in ["train_loss", "train_accs", "test_loss", "test_accs", "additional_data", "dataset_type"]
        },
    }
    for model, model_data in zip(loaded_models, loaded_model_data)
]
print(f"Reconstructed {len(all_models)} trained models")

Looking for models and dataset in: models_and_data
Found 10 model files:
  - model_additional_0.pt
  - model_additional_1693.pt
  - model_additional_2634.pt
  - model_additional_282.pt
  - model_additional_3104.pt
  - model_additional_3575.pt
  - model_additional_4516.pt
  - model_additional_5457.pt
  - model_additional_6398.pt
  - model_additional_752.pt
✅ Model loaded into provided instance from models_and_data\model_additional_0.pt
Successfully loaded: model_additional_0.pt
✅ Model loaded into provided instance from models_and_data\model_additional_1693.pt
Successfully loaded: model_additional_1693.pt
✅ Model loaded into provided instance from models_and_data\model_additional_2634.pt
Successfully loaded: model_additional_2634.pt
✅ Model loaded into provided instance from models_and_data\model_additional_282.pt
Successfully loaded: model_additional_282.pt
✅ Model loaded into provided instance from models_and_data\model_additional_3104.pt
Successfully loaded: model_additional_3104.pt


✅ Dataset loaded from models_and_data\dataset.pt
Dataset type: data
Dataset quantities: [0, 282, 752, 1693, 2634, 3104, 3575, 4516, 5457, 6398]

Tensor shapes:
  x_base_train: torch.Size([188, 194])
  y_base_train: torch.Size([188, 97])
  x_additional: torch.Size([6398, 194])
  y_additional: torch.Size([6398, 97])
  x_test: torch.Size([9409, 194])
  y_test: torch.Size([9409, 97])
Reconstructed 10 trained models


## Perturbations

Using the saved datasets, we perform model perturbations. 

In [5]:
device = 'cuda'

x_base_train = loaded_dataset['x_base_train'].to(device)
y_base_train = loaded_dataset['y_base_train'].to(device)
x_additional = loaded_dataset['x_additional'].to(device)
y_additional = loaded_dataset['y_additional'].to(device)
x_test = loaded_dataset['x_test'].to(device)
y_test = loaded_dataset['y_test'].to(device)

# Loss function and metrics already grabbed from the model module
analyze_wiggles_metrics(
    model_list = all_models, 
    x_base_train = x_base_train,
    y_base_train = y_base_train, 
    x_additional = x_additional,
    y_additional = y_additional,
    x_test = x_test,
    y_test = y_test, 
    dataset_quantities = dataset_quantities, 
    dataset_type = dataset_type, 
    metrics = {"loss": loss_fn, **other_metrics}, 
    coefficients = coefficients,
    num_directions = num_directions,
    perturbation_seed = perturbation_seed,
    base_output_dir = base_output_dir,
    device = device,  # can be set to GPU if needed
)


""" Our saved results are structured as follows:
wiggle_results: List of dictionaries containing wiggle test results
Each dictionary is of the form
{
'loss':
'coefficients':
'accs':
'perturbation_seed':
'perturbation_norm':
}
model: PyTorch model used in analysis (state_dict will be saved)
output_dir: Directory to save results (default: "imgs/swiss/random_dirs")
filename: Name of output file (default: "random_directions.npz")
**kwargs: Additional key-value pairs to be saved in the output file
Typically:
'additional_data':
'model_trained_data':
'dataset_type':
'base_dataset_size': 
'test_loss':
'test_accs':
'num_params':
"""

The number of parameters of the perturbation is 73097
Testing on data with 0 samples - 500 directions
Testing model trained on 0 additional data.
Loss: 0.0186
Accs: 0.0201


Wiggle completed in 46.61 seconds for data model trained with 0 samples
Saved to data_0

Testing model trained on 1693 additional data.
Loss: 0.0152
Accs: 0.2024


Wiggle completed in 48.43 seconds for data model trained with 1693 samples
Saved to data_0

Testing model trained on 2634 additional data.
Loss: 0.0122
Accs: 0.3052


Wiggle completed in 50.22 seconds for data model trained with 2634 samples
Saved to data_0

Testing model trained on 282 additional data.
Loss: 0.0240
Accs: 0.0505


Wiggle completed in 48.03 seconds for data model trained with 282 samples
Saved to data_0

Testing model trained on 3104 additional data.
Loss: 0.0110
Accs: 0.3715


...

Wiggle completed in 51.91 seconds for data model trained with 6398 samples
Saved to data_4516

Testing model trained on 752 additional data.
Testing on data with 5457 samples - 500 directions
Testing model trained on 0 additional data.
Testing model trained on 1693 additional data.
Testing model trained on 2634 additional data.
Testing model trained on 282 additional data.
Testing model trained on 3104 additional data.
Testing model trained on 3575 additional data.
Testing model trained on 4516 additional data.
Testing model trained on 5457 additional data.
Loss: 0.0036
Accs: 1.0000


Wiggle completed in 51.64 seconds for data model trained with 5457 samples
Saved to data_5457

Testing model trained on 6398 additional data.
Loss: 0.0023
Accs: 1.0000


Wiggle completed in 52.09 seconds for data model trained with 6398 samples
Saved to data_5457

Testing model trained on 752 additional data.
Testing on data with 6398 samples - 500 directions
Testing model trained on 0 additional data.
Testing model trained on 1693 additional data.
Testing model trained on 2634 additional data.
Testing model trained on 282 additional data.
Testing model trained on 3104 additional data.
Testing model trained on 3575 additional data.
Testing model trained on 4516 additional data.
Testing model trained on 5457 additional data.
Testing model trained on 6398 additional data.
Loss: 0.0023
Accs: 1.0000


Wiggle completed in 49.68 seconds for data model trained with 6398 samples
Saved to data_6398

Testing model trained on 752 additional data.


' Our saved results are structured as follows:\nwiggle_results: List of dictionaries containing wiggle test results\nEach dictionary is of the form\n{\n\'loss\':\n\'coefficients\':\n\'accs\':\n\'perturbation_seed\':\n\'perturbation_norm\':\n}\nmodel: PyTorch model used in analysis (state_dict will be saved)\noutput_dir: Directory to save results (default: "imgs/swiss/random_dirs")\nfilename: Name of output file (default: "random_directions.npz")\n**kwargs: Additional key-value pairs to be saved in the output file\nTypically:\n\'additional_data\':\n\'model_trained_data\':\n\'dataset_type\':\n\'base_dataset_size\': \n\'test_loss\':\n\'test_accs\':\n\'num_params\':\n'